# Phase 2: Feature Engineering Analysis

This notebook analyzes the feature vectors built from extracted requirements.

**What to look for:**
- Feature dimensionality by approach
- Embedding space visualization (pre-clustering)
- Natural groupings by division/function
- Comparison across feature approaches

In [ ]:
# Setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import json
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

from config import get_output_path
from src.archetypes.extraction import RequirementExtractor
from src.archetypes.feature_engineering import FeatureOutput

In [ ]:
# Load extraction results (for metadata)
extraction_path = get_output_path("archetypes", "phase_1_extraction", "extracted_requirements.json")
extractions = RequirementExtractor.load_results(str(extraction_path))
extraction_lookup = {e.jd_id: e for e in extractions}
print(f"✅ Loaded {len(extractions)} extractions")

# Build metadata DataFrame
metadata_records = []
for e in extractions:
    if e.extraction_success:
        record = {'jd_id': e.jd_id}
        record.update(e.metadata)
        # Add skill summary for hover
        record['top_skills'] = ', '.join(e.get_all_skills_flat()[:5])
        metadata_records.append(record)

metadata_df = pd.DataFrame(metadata_records)
print(f"✅ Built metadata for {len(metadata_df)} JDs")

In [ ]:
# Load feature outputs
feature_dir = get_output_path("archetypes", "phase_2_features")

feature_outputs = {}
if feature_dir.exists():
    for subdir in feature_dir.iterdir():
        if subdir.is_dir() and (subdir / "features.npy").exists():
            feature_outputs[subdir.name] = FeatureOutput.load(str(subdir))
            print(f"✅ Loaded features: {subdir.name} - shape {feature_outputs[subdir.name].features.shape}")

if not feature_outputs:
    print("❌ No feature outputs found. Run: python run_archetype_pipeline.py --features")

## 1. Feature Dimensionality Comparison

In [ ]:
# Compare feature dimensions
dim_data = []
for name, fo in feature_outputs.items():
    dim_data.append({
        'Experiment': name,
        'Approach': fo.config.approach.value,
        'Samples': fo.n_samples,
        'Feature Dim': fo.feature_dim,
        'Description': fo.config.description or '',
    })

dim_df = pd.DataFrame(dim_data)
display(dim_df)

if len(dim_df) > 1:
    fig = px.bar(dim_df, x='Experiment', y='Feature Dim', color='Approach',
                 title='Feature Dimensionality by Approach',
                 hover_data=['Description'])
    fig.update_layout(height=400)
    fig.show()

## 2. UMAP Visualization (Pre-Clustering)

Visualize the embedding space before clustering to see if natural groupings exist.

In [ ]:
# Compute UMAP for each feature output
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    print("⚠️ umap-learn not installed. Install with: pip install umap-learn")
    UMAP_AVAILABLE = False

umap_results = {}

if UMAP_AVAILABLE:
    for name, fo in feature_outputs.items():
        print(f"Computing UMAP for {name}...")
        reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
        embedding_2d = reducer.fit_transform(fo.features)
        
        # Build DataFrame with coordinates and metadata
        umap_df = pd.DataFrame({
            'jd_id': fo.ids,
            'x': embedding_2d[:, 0],
            'y': embedding_2d[:, 1],
        })
        
        # Merge with metadata
        umap_df = umap_df.merge(metadata_df, on='jd_id', how='left')
        umap_results[name] = umap_df
        
    print(f"✅ Computed UMAP for {len(umap_results)} feature sets")

In [ ]:
# Interactive UMAP viewer
if UMAP_AVAILABLE and umap_results:
    # Dropdown for feature set
    feature_dropdown = widgets.Dropdown(
        options=list(umap_results.keys()),
        description='Feature Set:',
        style={'description_width': 'initial'}
    )
    
    # Dropdown for color
    available_colors = ['division', 'function', 'level', 'org_unit', 'department']
    color_options = [c for c in available_colors if c in metadata_df.columns]
    color_options = ['(none)'] + color_options
    
    color_dropdown = widgets.Dropdown(
        options=color_options,
        value=color_options[1] if len(color_options) > 1 else '(none)',
        description='Color by:',
        style={'description_width': 'initial'}
    )
    
    output = widgets.Output()
    
    def update_umap(change):
        with output:
            output.clear_output(wait=True)
            
            feature_name = feature_dropdown.value
            color_by = color_dropdown.value
            
            df = umap_results[feature_name].copy()
            
            # Build hover text
            hover_cols = ['jd_id']
            if 'title' in df.columns:
                hover_cols.append('title')
            if 'top_skills' in df.columns:
                hover_cols.append('top_skills')
            
            if color_by == '(none)':
                fig = px.scatter(
                    df, x='x', y='y',
                    hover_data=hover_cols,
                    title=f'UMAP: {feature_name}'
                )
            else:
                fig = px.scatter(
                    df, x='x', y='y',
                    color=color_by,
                    hover_data=hover_cols,
                    title=f'UMAP: {feature_name} (colored by {color_by})'
                )
            
            fig.update_layout(height=600, width=800)
            fig.update_traces(marker=dict(size=5, opacity=0.7))
            fig.show()
    
    feature_dropdown.observe(update_umap, names='value')
    color_dropdown.observe(update_umap, names='value')
    
    display(widgets.HBox([feature_dropdown, color_dropdown]))
    display(output)
    
    # Initial render
    update_umap(None)

## 3. Feature Approach Comparison

Side-by-side comparison of how different feature approaches affect the embedding space.

In [ ]:
# Side by side UMAP comparison
if UMAP_AVAILABLE and len(umap_results) >= 2:
    # Get first two feature sets for comparison
    names = list(umap_results.keys())[:4]  # Max 4 for 2x2 grid
    
    n_plots = len(names)
    cols = 2
    rows = (n_plots + 1) // 2
    
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=names
    )
    
    # Determine color field
    color_field = 'division' if 'division' in metadata_df.columns else None
    
    for i, name in enumerate(names):
        row = i // cols + 1
        col = i % cols + 1
        
        df = umap_results[name]
        
        if color_field and color_field in df.columns:
            for category in df[color_field].unique():
                mask = df[color_field] == category
                fig.add_trace(
                    go.Scatter(
                        x=df.loc[mask, 'x'],
                        y=df.loc[mask, 'y'],
                        mode='markers',
                        name=str(category),
                        marker=dict(size=4, opacity=0.6),
                        showlegend=(i == 0),  # Only show legend for first subplot
                    ),
                    row=row, col=col
                )
        else:
            fig.add_trace(
                go.Scatter(
                    x=df['x'],
                    y=df['y'],
                    mode='markers',
                    marker=dict(size=4, opacity=0.6),
                    showlegend=False,
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=400 * rows,
        width=900,
        title='Feature Approach Comparison (UMAP)'
    )
    fig.show()
    
    print("\n💡 Look for:")
    print("   - Do colors (divisions) naturally cluster together?")
    print("   - Which approach shows clearer separation?")
    print("   - Are there outliers far from main clusters?")

## 4. Categorical Feature Coverage (for Structured Approach)

In [ ]:
# Show categorical mappings for structured approaches
for name, fo in feature_outputs.items():
    if fo.categorical_mappings:
        print(f"\n📊 {name} - Categorical Feature Mappings:")
        print("="*50)
        
        for field, mapping in fo.categorical_mappings.items():
            print(f"\n{field.upper()} ({len(mapping)} unique values):")
            for value, idx in sorted(mapping.items(), key=lambda x: x[1]):
                print(f"   [{idx}] {value}")

## 5. Feature Statistics

In [ ]:
# Basic feature statistics
for name, fo in feature_outputs.items():
    print(f"\n📈 {name} Feature Statistics:")
    print("="*50)
    
    features = fo.features
    
    # Check for zero vectors
    norms = np.linalg.norm(features, axis=1)
    zero_vectors = np.sum(norms < 1e-6)
    
    print(f"   Shape: {features.shape}")
    print(f"   Mean: {features.mean():.4f}")
    print(f"   Std: {features.std():.4f}")
    print(f"   Min: {features.min():.4f}")
    print(f"   Max: {features.max():.4f}")
    print(f"   Zero vectors: {zero_vectors} ({zero_vectors/len(features):.1%})")
    print(f"   Avg L2 norm: {norms.mean():.4f}")

## 6. Division/Function Distribution in Feature Space

In [ ]:
# Show distribution of metadata fields
if 'division' in metadata_df.columns:
    fig = px.pie(metadata_df, names='division', title='Distribution by Division')
    fig.update_layout(height=400, width=500)
    fig.show()

if 'function' in metadata_df.columns:
    func_counts = metadata_df['function'].value_counts().head(15)
    fig = px.bar(x=func_counts.index, y=func_counts.values, 
                 title='Top 15 Functions',
                 labels={'x': 'Function', 'y': 'Count'})
    fig.update_layout(height=400, width=700)
    fig.show()

## 7. Summary & Recommendations

In [ ]:
print("\n" + "="*60)
print("PHASE 2 SUMMARY")
print("="*60)

print(f"\n📊 Feature Outputs Created: {len(feature_outputs)}")
for name, fo in feature_outputs.items():
    print(f"   - {name}: {fo.features.shape} ({fo.config.approach.value})")

print("\n💡 Questions to Consider:")
print("   1. Do JDs from the same division cluster together naturally?")
print("   2. Which feature approach shows clearest separation?")
print("   3. Are there obvious outliers that might become noise?")
print("   4. Should division/function be explicitly included in features?")

print("\n➡️ Next: Run Phase 3 (Clustering) to see how these features cluster")